In [27]:
import xarray as xr
import numpy as np
import geopandas as gpd
import random
import matplotlib.pyplot as plt


AttributeError: module 'matplotlib' has no attribute 'get_data_path'

In [26]:
#Hourly 2D bottom currents
current_jan_jun = xr.open_dataset("/cluster/projects/itk-SINMOD/coral-mapping/midnor/samp_2D_jan_jun.nc")
current_jun_aug = xr.open_dataset("/cluster/projects/itk-SINMOD/coral-mapping/midnor/samp_2D_jun-aug.nc")
current_sep_dec = xr.open_dataset("/cluster/projects/itk-SINMOD/coral-mapping/midnor/samp_2D_sep-dec.nc")

In [3]:
#Remove overlap
current_jan_jun = current_jan_jun.isel(time=slice(0, -360))  
current_jun_aug = current_jun_aug.isel(time=slice(0, -192))  

In [ ]:
#Merge (not possible with this memory size.. Can do this later if necessary)
current_2019 = xr.concat([current_jan_jun, current_jun_aug, current_sep_dec], dim="time")

In [5]:
print(list(current_jan_jun.variables))

['time', 'grid_mapping', 'LayerDepths', 'xc', 'yc', 'zc', 'depth', 'DXxDYy', 'gridLats', 'gridLons', 'u_velocity', 'v_velocity', 'temperature', 'salinity', 'elevation', 'u-wind', 'v-wind', 'u-bottom', 'v-bottom', 'ice_thickness', 'ice_compactness', 'u_ice', 'v_ice', 'salinity_ice', 'ice_melt_rate', 'ice_prod_rate', 'IceVFluxX', 'IceVFluxY', 'IceAFluxX', 'IceAFluxY']


In [7]:
u_vel_jan_jun = current_jan_jun.variables['u_velocity']
u_vel_jun_aug = current_jun_aug.variables['u_velocity']
u_vel_sep_dec = current_sep_dec.variables['u_velocity']

In [36]:
combined_u_vel = xr.concat([u_vel_jan_jun, u_vel_jun_aug, u_vel_sep_dec], dim='time')


TypeError: can only concatenate xarray Dataset and DataArray objects, got <class 'xarray.core.variable.Variable'>

In [8]:
v_vel_jan_jun = current_jan_jun.variables['v_velocity']
v_vel_jun_aug = current_jun_aug.variables['v_velocity']
v_vel_sep_dec = current_sep_dec.variables['v_velocity']

In [10]:
u_vel_bottom_jan_jun = current_jan_jun.variables['u-bottom']
u_vel_bottom_jun_aug = current_jun_aug.variables['u-bottom']
u_vel_bottom_sep_dec = current_sep_dec.variables['u-bottom']

In [11]:
v_vel_bottom_jan_jun = current_jan_jun.variables['v-bottom']
v_vel_bottom_jun_aug = current_jun_aug.variables['v-bottom']
v_vel_bottom_sep_dec = current_sep_dec.variables['v-bottom']

In [19]:
#Coral data
import pandas as pd

coral_data = pd.read_parquet('/cluster/home/maikents/midnor_total_coral_data_processed_v2.parquet')


In [34]:
#Histogram for current speed
hor_res = current_jan_jun.grid_mapping.attrs['horizontal_resolution']
t_start, t_stop = (0, (len(current_jan_jun.variables['time'])+len(current_jun_aug.variables['time'])+len(current_sep_dec.variables['time'])))
coral_values = []
valid_coords = []
xc_max = current_jan_jun.variables['xc'].shape[0]
yc_max = current_jan_jun.variables['yc'].shape[0]
for time in range(t_start, t_stop):

        for x, y in zip(coral_data['x']/hor_res, coral_data['y']/hor_res):
            
            #Ensure the coordinates are within the grid bounds
            if 0 <= x <= xc_max and 0 <= y <= yc_max:

                depth_at_point = netcdf_data.variables['depth'][int(y), int(x)]
                layer_index = np.searchsorted(cumulative_depth, depth_at_point)

                current_speed_jan_jun = np.sqrt(u_vel_jan_jun[time, layer_index, int(y), int(x)]**2 + v_vel_jan_jun[time, layer_index, int(y), int(x)]**2)
                #value = netcdf_data.variables[variable_name][time, layer_index, int(y), int(x)]

                if not np.ma.is_masked(value):
                    coral_values.append(value)
                    if (x, y) not in valid_coordinates:
                        valid_coordinates.append((x, y))


In [ ]:
#Plot histograms
import random
import numpy as np
import matplotlib.pyplot as plt

def plot_histogram_at_coral_points(netcdf_data, coral_data, variable_name, t_range, generate_random=False, save=False, save_path="midnor"):

    zc = netcdf_data.variables['LayerDepths'][:]
    cumulative_depth = np.cumsum(zc, axis=0)

    xc_max = netcdf_data.variables['xc'].shape[0]
    yc_max = netcdf_data.variables['yc'].shape[0]

    hor_res = netcdf_data.variables['grid_mapping'].getncattr('horizontal_resolution')

    t_start, t_stop = t_range

    if t_stop == -1:
        t_stop = netcdf_data.variables['time'].shape[0]

    # Extract variable values at coral data points
    coral_values = []
    valid_coordinates = []

    for time in range(t_start, t_stop):

        for x, y in zip(coral_data['x']/hor_res, coral_data['y']/hor_res):
            
            # Ensure the coordinates are within the grid bounds
            if 0 <= x <= xc_max and 0 <= y <= yc_max:

                depth_at_point = netcdf_data.variables['depth'][int(y), int(x)]
                layer_index = np.searchsorted(cumulative_depth, depth_at_point)

                if variable_name == 'current_speed':
                    value = np.sqrt(netcdf_data.variables['u_velocity'][time, layer_index, int(y), int(x)]**2 + netcdf_data.variables['v_velocity'][time, layer_index, int(y), int(x)]**2)
                else:
                    value = netcdf_data.variables[variable_name][time, layer_index, int(y), int(x)]

                if not np.ma.is_masked(value):
                    coral_values.append(value)
                    if (x, y) not in valid_coordinates:
                        valid_coordinates.append((x, y))

    # Convert list to numpy array for easier manipulation
    coral_values = np.array(coral_values)

    # Flatten the array to combine all time steps
    coral_values_flat = coral_values.flatten()

    # Calculate statistics for coral values
    stats = {
        'mean': np.mean(coral_values_flat),
        'max': np.max(coral_values_flat),
        'min': np.min(coral_values_flat),
        '90th_percentile': np.percentile(coral_values_flat, 90),
        '10th_percentile': np.percentile(coral_values_flat, 10)
    }

    print(f"Statistics for coral values: {stats}")

    # Determine the bin edges for the histograms
    bin_edges = np.histogram_bin_edges(coral_values_flat, bins=20)

    # Plot histogram of the combined variable values
    plt.hist(coral_values_flat, bins=bin_edges, edgecolor='black')
    plt.xlabel(f'{variable_name.capitalize()}')
    plt.ylabel('Frequency')
    plt.title(f'Histogram of {variable_name.capitalize()} at Coral Data Points - {save_path.capitalize()}')
    
    if save:
        plt.savefig(f'plots/variable_histograms/{variable_name}_histogram_coral_points_{save_path}.png')
    
    plt.show()

    print(f"Number of valid coordinates: {len(valid_coordinates)} out of {len(coral_data)}")

    if generate_random:
        # Plotting random histogram

        no_points = int(len(valid_coordinates))

        # Generate same number of valid co-ords as random x and y locations within the grid
        valid_xc_yc_indices = np.argwhere(np.logical_and(~netcdf_data.variables['temperature'][0,0].mask, netcdf_data.variables['depth'][:] <=275,  netcdf_data.variables['depth'][:] >= 150))

        random_values = []

        for time in range(t_start, t_stop):

            selected_indices = valid_xc_yc_indices[np.random.choice(valid_xc_yc_indices.shape[0], size=no_points, replace=False)]

            for y, x in selected_indices:
                
                # Ensure the coordinates are within the grid bounds
                if 0 <= x <= xc_max and 0 <= y <= yc_max:
                    depth_at_point = netcdf_data.variables['depth'][int(y), int(x)]
                    layer_index = np.searchsorted(cumulative_depth, depth_at_point)

                    if variable_name == 'current_speed':
                        value = np.sqrt(netcdf_data.variables['u_velocity'][time, layer_index, int(y), int(x)]**2 + netcdf_data.variables['v_velocity'][time, layer_index, int(y), int(x)]**2)
                    else:
                        value = netcdf_data.variables[variable_name][time, layer_index, int(y), int(x)]

                    if not np.ma.is_masked(value):
                        random_values.append(value)
        
        # Convert list to numpy array for easier manipulation
        random_values = np.array(random_values)

        # Flatten the array to combine all time steps
        random_values_flat = random_values.flatten()

        # Plot histogram of the combined random variable values
        plt.hist(random_values_flat, bins=bin_edges, edgecolor='black')
        plt.xlabel(f'Random {variable_name.capitalize()}')
        plt.ylabel('Frequency')
        plt.title(f'Histogram of Random {variable_name.capitalize()} Values - {save_path.capitalize()}')
        
        if save:
            plt.savefig(f'plots/variable_histograms/{variable_name}_histogram_coral_points_{save_path}_random.png')
        
        plt.show()

        random_stats = {
            'mean': np.mean(random_values_flat),
            'max': np.max(random_values_flat),
            'min': np.min(random_values_flat),
            '90th_percentile': np.percentile(random_values_flat, 90),
            '10th_percentile': np.percentile(random_values_flat, 10)
        }

        print(f"Statistics for random values: {random_stats}")

        return coral_values, stats, random_values, random_stats
    return coral_values, stats

# Example usage
# plot_histogram_at_coral_points(PhysStates_data, coral_data, 'temperature', (0, 50))
# plot_histogram_at_coral_points(nor4km_PhysStates_data, nor4km_coral_data, 'temperature', (0, 50))
midnor_temp, midnor_stats, midnor_temp_random, midnor_random_stats = plot_histogram_at_coral_points(midnor_PhysStates_data, midnor_coral_data, 'temperature', (0, 10), True)
# plot_histogram_at_coral_points(midnor_PhysStates_data, midnor_coral_data, 'salinity', (0, 50))